# அமர்வு 2 – குறைந்த RAG குழாய்

Foundry Local + sentence-transformers embeddings பயன்படுத்தி எளிய Retrieval-Augmented Generation குழாயை உருவாக்குங்கள்.


### விளக்கம்: சார்பு நிறுவல்
இந்த குழாய்க்கு குறைந்தபட்ச தொகுப்புகளை நிறுவுகிறது:
- `foundry-local-sdk` உள்ளூர் மாடல் மேலாண்மைக்காக (தெளிவான BASE_URL பாதையை பயன்படுத்தவில்லை என்றால்).
- `openai` இணக்கமான SDK அமைப்புகளுக்காக (சில பயன்பாடுகள்).
- `sentence-transformers` எம்பெடிங்குகளுக்காக.
- `numpy` வெக்டர் கணிதத்திற்காக.
மீண்டும் இயக்குவதற்கு பாதுகாப்பானது; சூழல் ஏற்கனவே பூர்த்தி செய்யப்பட்டிருந்தால் தவிர்க்கவும்.


# சூழல்
இந்த நோட்புக் ஒரு மிகச்சிறிய Retrieval-Augmented Generation (RAG) குழாய்வழியை முழுமையாக உள்ளூரில் இயங்குவதற்காக உருவாக்குகிறது:
- Foundry Local மாடலுடன் இணைக்கிறது (SDK அல்லது BASE_URL மூலம் தானாக கண்டறிகிறது).
- ஒரு சிறிய நினைவக ஆவணக் தொகுப்பை உருவாக்கி, அதை Sentence Transformers மூலம் எம்பெட்ஸ் செய்கிறது.
- எளிய வெக்டர் ஒற்றுமை தேடலை (வெளியக குறியீடு இல்லாமல்) வெளிப்படையாக செயல்படுத்துகிறது.
- பல HTTP fallback வழிகளின் மூலம் நிலையான உருவாக்க கோரிக்கைகளை வெளியிடுகிறது (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`).
- ஆரம்ப முயற்சிகள் தோல்வியடைந்தால் மாற்று மாடல் வடிவங்களை மீண்டும் முயற்சிக்கும் `answer()` உதவியாளரை வழங்குகிறது.

பெரிய ஆவணத் தொகுப்புகள், நிலையான வெக்டர் சேமிப்புகள், அல்லது மதிப்பீட்டு அளவுகோல்கள் (RAG மதிப்பீட்டு நோட்புக் காண்க) ஆகியவற்றுக்கு மாறுவதற்கு முன் இதை ஒரு கண்டறிதல் வார்ப்புருவாக பயன்படுத்தவும்.


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### விளக்கம்: முக்கிய இறக்குமுதல்கள்
எம்பெடிங் மற்றும் உள்ளூர் தீர்மானத்திற்கான முக்கிய நூலகங்களை ஏற்றுகிறது:
- SentenceTransformer, அடர்த்தியான வெக்டர் எம்பெடிங்குகளுக்காக.
- FoundryLocalManager (விருப்பத்தேர்வு), உள்ளூர் சேவையை நிர்வகிக்க.
- OpenAI client, பரிச்சயமான பொருள் வடிவங்களுக்காக (பின்னர் HTTP நேரடியாகப் பயன்படுத்தினாலும்).


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### விளக்கம்: டாய் டாக்குமென்ட் கார்பஸ்  
டொமைன் அறிக்கைகளின் சிறிய இன்மெமரி பட்டியலை வரையறுக்கிறது. மீள்பார்வையை வேகமாகவும் கட்டுப்பாட்டுடனும் வைத்திருப்பதன் மூலம், தரவுகளை கையாள்வதற்குப் பதிலாக பைப்லைன் செயல்முறைகளின் (மீட்பு + நிலைநிறுத்தல்) மீது கவனம் செலுத்துகிறது.


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### விளக்கம்: இணைப்பு, மாடல் தேர்வு & எம்பெடிங் தொடக்கம்
வலுவான இணைப்பு தார்மீகங்கள்:
1. விருப்பமாக `BASE_URL` (தூய HTTP பாதை) பயன்படுத்துகிறது, இல்லையெனில் FoundryLocalManager-க்கு மாறுகிறது.
2. `/v1/models` ஐ சோதித்து, சிறந்த பொருந்தும் மConcrete மாடல் ஐடி (சரியான alias > canonical குடும்பம் > முதல் கிடைக்கும்) தேர்வு செய்கிறது.
3. `FOUNDRY_CONNECT_RETRIES` மற்றும் தாமதத்துடன் கட்டமைக்கக்கூடிய retry loop ஐ செயல்படுத்துகிறது.
4. toy corpus க்கான SentenceTransformer embeddings (சீரமைக்கப்பட்ட vectors) ஐ தொடங்குகிறது.
5. OpenAI SDK பதிப்பை மீண்டும் உருவாக்கத்திற்காகப் பதிவு செய்கிறது.
சேவை இல்லையெனில், அது செயலிழக்காமல் அதை தொடங்க வழிகாட்டுதலை அச்சிடுகிறது.


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### விளக்கம்: Retrieve செயல்பாடு (வெக்டர் ஒற்றுமை)
`retrieve(query, k=3)` கேள்வியை குறியாக்கி, கோசைன் ஒற்றுமையை (சீரமைக்கப்பட்ட வெக்டர்களில் டாட் பொருத்தம்) கணக்கிடுகிறது மற்றும் சிறந்த-k ஆவண குறியீடுகளை திருப்புகிறது. இது தெளிவுக்காக குறைந்தபட்சமாகவும் நினைவில் வைத்துக்கொள்ளக்கூடியதாகவும் உள்ளது.


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### விளக்கம்: SDK அடிப்படையிலான உருவாக்கம் & பதில் உதவியாளர்
Foundry Local SDK மற்றும் OpenAI-இன் இணக்கமான client முறைகளை பயன்படுத்துவதற்காக மறுசீரமைக்கப்பட்டது, கையேடு மூலமான HTTP பதிவுகளை மாற்றி:
- முதன்மை பாதை: `client.chat.completions.create` (கட்டமைக்கப்பட்ட செய்திகள்).
- மாற்று வழிகள்: `client.completions.create` (பழைய prompt) மற்றும் `client.responses.create` (சரளமான பதில்கள் API).
- மாறுபட்ட மாடல் அடையாளங்களை (RAW மற்றும் stripped ALT) சாதாரணமாக்கி, இணக்கத்தன்மையை விரிவுபடுத்துகிறது.
- `answer()` மேல்நிலை-கே (top-k) மீட்கப்பட்ட ஆவணங்களிலிருந்து ஒரு நிலையான prompt-ஐ உருவாக்கி, முயற்சிகளின் வரிசைப்படுத்தப்பட்ட தடங்களை பதிவு செய்கிறது.
இது OpenAI-இன் இணக்கமான முறைகளில் மாறுபாடுகளை சமாளிக்க, தரமான செயல்திறனுடன் வாசிக்க எளிதான தர்க்கத்தை வழங்குகிறது.


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### விளக்கம்: பேட்ச் கேள்வி ச்மோக் டெஸ்ட்
`answer()` மூலம் பல முக்கிய பயனர் கேள்விகளை செயல்படுத்துகிறது, இதன் மூலம் சரிபார்க்கப்படுகிறது:
- மீட்பு குறியீடுகள் நம்பகமான ஆதார ஆவணங்களுக்கு பொருந்துகின்றன.
- பின்வாங்கும் வழிமுறைகள் செயல்படுகின்றன (வழிமுறை மதிப்பு 'failed' அல்ல).
- பதில்கள் அடிப்படை வழிகாட்டுதலை மதிக்கின்றன (தவறான தகவல்கள் இல்லை).
கடைசி முடிவு பொருளை தற்செயலான ஆய்வுக்காக பதிவு செய்கிறது.


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### விளக்கம்: ஒற்றை பதில் வசதியான அழைப்பு
எளிதாக நகலெடுத்து ஒட்ட அல்லது பின்னர் குறிப்பிட்டுக் கொள்ள ஒரு இறுதி விரைவான ஒற்றை கேள்வி அழைப்பு. முன்பே செய்யப்பட்ட கேள்விகளுக்குப் பிறகு `answer()` ஐ idempotent முறையில் பயன்படுத்துவதை விளக்குகிறது.


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
